# Builds off of ML_Project_Jon but incorporates 311 data


In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline

In [11]:
pd.set_option('max_rows', 10)

## Load health inspection with 311 complaints

In [14]:
violations_df = pd.read_csv('vdf_with_complaints.csv', index_col=0) # file is in GitHub

### Remove columns we don't want in our training set

In [29]:
violations_df2 = violations_df.drop(['address', 'crit_violations', 'earliest_inspection',
                                     'latest_inspection', 'second_latest_inspection',
                                     'non_crit_violations', 'num_inspections',
                                     'crit_violations_recent_inspect', 'non_crit_violations_recent_inspect',
                                     'address2','latitude','longitude'],axis=1)
violations_df2.head()

,zipcode,boro,cuisine,crit_violations_train,non_crit_violations_train,num_inspections_train,average_crit_v_train,average_non_crit_v_train,time_since_last_inspection,time_since_first_inspection,crit_v_2plus,food_poisoning_complaints,food_establishment_complaints,electric_complaints,safety_complaints,rodent_complaints,dirty_conditions_complaints,missed_collection_complaints
40849391,10003,MANHATTAN,Japanese,10.0,7.0,5,2.00,1.40,500,953,0,17,47,2,1,24.0,33.0,31.0
41098999,10469,BRONX,Hamburgers,5.0,6.0,4,1.25,1.50,385,946,0,3,3,10,1,8.0,16.0,7.0
41140992,11369,QUEENS,Sandwiches,2.0,5.0,4,0.50,1.25,398,916,0,0,1,0,0,0.0,0.0,0.0
41317857,10017,MANHATTAN,Café/Coffee/Tea,5.0,6.0,5,1.00,1.20,385,740,0,26,60,3,2,12.0,13.0,9.0
41382811,10469,BRONX,Caribbean,9.0,9.0,5,1.80,1.80,19,1008,0,0,3,10,0,6.0,31.0,13.0


In [30]:
#violations_df2 = (violations_df2 - violations_df2.mean()) / (violations_df2.max() - violations_df2['food_poisoning_complaints'].min()) 


ValueError: could not convert string to float: American

### Convert categorical variables to dummy variables

In [31]:
violations_df3 = pd.get_dummies(violations_df2)

In [34]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()

violations_df4= scale.fit_transform(violations_df3.as_matrix())

### Separate target variable

In [17]:
X = violations_df3.drop(['crit_v_2plus'],axis=1)
y = violations_df3['crit_v_2plus']

### Split into training and test sets

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 
X_train

,zipcode,crit_violations_train,non_crit_violations_train,num_inspections_train,average_crit_v_train,average_non_crit_v_train,time_since_last_inspection,time_since_first_inspection,food_poisoning_complaints,food_establishment_complaints,...,cuisine_Soups & Sandwiches,cuisine_Southwestern,cuisine_Spanish,cuisine_Steak,cuisine_Tapas,cuisine_Tex-Mex,cuisine_Thai,cuisine_Turkish,cuisine_Vegetarian,cuisine_Vietnamese/Cambodian/Malaysia
41538262,10314,8.0,7.0,5,1.60,1.400000,294,955,0,10,...,0,0,0,0,0,0,0,0,0,0
40959910,11691,6.0,6.0,5,1.20,1.200000,355,1075,2,7,...,0,0,0,0,0,0,0,0,0,0
50000280,10458,3.0,7.0,4,0.75,1.750000,211,929,5,16,...,0,0,0,0,0,0,0,0,0,0
40402095,10020,4.0,6.0,4,1.00,1.500000,400,1078,18,48,...,0,0,0,0,0,0,0,0,0,0
50001610,11206,9.0,6.0,6,1.50,1.000000,13,853,1,2,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41701184,10459,7.0,10.0,7,1.00,1.428571,191,882,6,9,...,0,0,0,0,0,0,0,0,0,0
50036254,10036,0.0,1.0,1,0.00,1.000000,374,374,4,9,...,0,0,0,0,0,0,0,0,0,0
41383581,11428,20.0,19.0,10,2.00,1.900000,209,999,0,0,...,0,0,0,0,0,0,0,0,0,0
40513031,10024,2.0,3.0,2,1.00,1.500000,421,770,3,15,...,0,0,0,0,0,0,0,0,0,0


#### Calculate metrics of predicting 0 class for all

In [19]:
pred_zero = np.zeros(len(y_test))
print('accuracy = ' + str(sklearn.metrics.accuracy_score(y_test, pred_zero)))
print('recall = ' + str(sklearn.metrics.recall_score(y_test, pred_zero)))
print('precision = ' + str(sklearn.metrics.precision_score(y_test, pred_zero)))
print('f1 = ' + str(sklearn.metrics.f1_score(y_test, pred_zero)))

accuracy = 0.664948453608
recall = 0.0
precision = 0.0
f1 = 0.0


/Users/motorrecoveryresearchlab/anaconda/envs/py27/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/motorrecoveryresearchlab/anaconda/envs/py27/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


### Logistic Regression

#### w/ L2-regularization

In [20]:
lr = linear_model.LogisticRegression(penalty = 'l2',C=1e20)
lr.fit(X_train, y_train)
pred_lr = lr.predict(X_test)
print('accuracy = ' + str(sklearn.metrics.accuracy_score(y_test, pred_lr)))
print('recall = ' + str(sklearn.metrics.recall_score(y_test, pred_lr)))
print('precision = ' + str(sklearn.metrics.precision_score(y_test, pred_lr)))
print('f1 = ' + str(sklearn.metrics.f1_score(y_test, pred_lr)))

accuracy = 0.672120125504
recall = 0.127759197324
precision = 0.545714285714
f1 = 0.207046070461


#### w/ L1-regularization

In [21]:
lr = linear_model.LogisticRegression(penalty = 'l1',C=1e20)
lr.fit(X_train, y_train)
pred_lr = lr.predict(X_test)
print('accuracy = ' + str(sklearn.metrics.accuracy_score(y_test, pred_lr)))
print('recall = ' + str(sklearn.metrics.recall_score(y_test, pred_lr)))
print('precision = ' + str(sklearn.metrics.precision_score(y_test, pred_lr)))
print('f1 = ' + str(sklearn.metrics.f1_score(y_test, pred_lr)))

accuracy = 0.668758404303
recall = 0.161872909699
precision = 0.518201284797
f1 = 0.246687054027


### Naive Bayes

#### w/ fit prior

In [22]:
nb = BernoulliNB(alpha=1,fit_prior=True)
nb.fit(X_train, y_train)
pred_nb = nb.predict(X_test)
print('accuracy = ' + str(sklearn.metrics.accuracy_score(y_test, pred_nb)))
print('recall = ' + str(sklearn.metrics.recall_score(y_test, pred_nb)))
print('precision = ' + str(sklearn.metrics.precision_score(y_test, pred_nb)))
print('f1 = ' + str(sklearn.metrics.f1_score(y_test, pred_nb)))

accuracy = 0.657776781712
recall = 0.0588628762542
precision = 0.423076923077
f1 = 0.103347034645


#### w/ uniform prior

In [23]:
nb = BernoulliNB(alpha=1,fit_prior=False)
nb.fit(X_train, y_train)
pred_nb = nb.predict(X_test)
print('accuracy = ' + str(sklearn.metrics.accuracy_score(y_test, pred_nb)))
print('recall = ' + str(sklearn.metrics.recall_score(y_test, pred_nb)))
print('precision = ' + str(sklearn.metrics.precision_score(y_test, pred_nb)))
print('f1 = ' + str(sklearn.metrics.f1_score(y_test, pred_nb)))

accuracy = 0.532496638279
recall = 0.645484949833
precision = 0.382784609282
f1 = 0.480577689243


### Random Forest

#### 10 trees

In [24]:
rf = RandomForestClassifier(n_estimators=10)
rf.fit(X_train, y_train)
pred_rf = rf.predict(X_test)
print('accuracy = ' + str(sklearn.metrics.accuracy_score(y_test, pred_rf)))
print('recall = ' + str(sklearn.metrics.recall_score(y_test, pred_rf)))
print('precision = ' + str(sklearn.metrics.precision_score(y_test, pred_rf)))
print('f1 = ' + str(sklearn.metrics.f1_score(y_test, pred_rf)))

accuracy = 0.651949798297
recall = 0.197993311037
precision = 0.455384615385
f1 = 0.275990675991


#### 20 trees

In [25]:
rf = RandomForestClassifier(n_estimators=20)
rf.fit(X_train, y_train)
pred_rf = rf.predict(X_test)
print('accuracy = ' + str(sklearn.metrics.accuracy_score(y_test, pred_rf)))
print('recall = ' + str(sklearn.metrics.recall_score(y_test, pred_rf)))
print('precision = ' + str(sklearn.metrics.precision_score(y_test, pred_rf)))
print('f1 = ' + str(sklearn.metrics.f1_score(y_test, pred_rf)))

accuracy = 0.657776781712
recall = 0.19397993311
precision = 0.47385620915
f1 = 0.275272899858
